In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-with-features-csv/Deeper Regression Features (1).csv


In [63]:
import torch
import torch.nn as nn
import torch.optim as optim

In [64]:
import matplotlib.pyplot as plt


In [65]:
file_path="/kaggle/input/data-with-features-csv/Deeper Regression Features (1).csv"
pd_read=pd.read_csv(file_path)
print(f"Dataset_shape{pd_read.shape}\n")

Dataset_shape(100, 4)



In [66]:
rows_to_display=15
print(pd_read.head(rows_to_display))

    distance_miles  time_of_day_hours  is_weekend  delivery_time_minutes
0             1.60               8.20           0                   7.22
1            13.09              16.80           1                  32.41
2             6.97               8.02           1                  17.47
3            10.66              16.07           0                  37.17
4            18.24              13.47           0                  38.36
5             5.74              16.59           0                  29.06
6             8.80              12.25           0                  23.94
7            15.36              11.76           1                  32.40
8             5.35               9.42           0                  17.06
9             2.46              14.44           0                  14.09
10            6.51               8.00           0                  33.38
11            4.06               9.33           1                  17.38
12           18.66              14.86           1  

In [67]:
import seaborn as sns

In [68]:
sample_tensor=torch.tensor([
     # distance, time_of_day, is_weekend, delivery_time
    [1.60,      8.20,        0,          7.22],   # row 1
    [13.09,     16.80,       1,          32.41],  # row 2       
    [6.97,      8.02,        1,          17.47],  # row 3
    [10.66,     16.07,       0,          37.17],  # row 4
    [18.24,     13.47,       0,          38.36]   # row 5
],dtype=torch.float32)

In [69]:
sample_hours=sample_tensor[:,1]
sample_weekends=sample_tensor[:,2]

print(f"Sample hours: {sample_hours}")
print(f"Sample Weekends: {sample_weekends}")

Sample hours: tensor([ 8.2000, 16.8000,  8.0200, 16.0700, 13.4700])
Sample Weekends: tensor([0., 1., 1., 0., 0.])


**# **As there are the two shifts in the weekdays we can there is late probably due to the high traffic so we need to make that different time slots so first we test the function on the limited time slot testing on the above time slots****

In [70]:
def rush_hours_features(hours_tensor,weekends_tensor):
    ## Morning rush is in between 8 to 10
    ## And Evening rush is in between 16 to 19
    ## Here we cant use the basic python functionalities like "if" can't use and the chaining of the boolen symbols can use
    is_morning_rush=(8.0>=hours_tensor) & (hours_tensor<10)
    is_evening_rush=(16.0>=hours_tensor) & (hours_tensor<19)
    is_weekday=(weekends_tensor==0)
    is_rush_hour_mask=is_weekday & (is_evening_rush | is_morning_rush)
    return is_rush_hour_mask.float()



In [71]:
rush_hour_sample=rush_hours_features(sample_hours,sample_weekends)

print(f"Sample Hour: {sample_hours.numpy()}")
print(f"Sample Weekday: {sample_weekends.numpy()}")
print(f"Rush_hour_on_sample: {rush_hour_sample.numpy()}")

Sample Hour: [ 8.2  16.8   8.02 16.07 13.47]
Sample Weekday: [0. 1. 1. 0. 0.]
Rush_hour_on_sample: [1. 0. 0. 0. 1.]


In [83]:
def prepare_data(df):
    # Extarcting all the data from the dataframe in the numpy array formate
    all_values=df.values
    
    # Converting all the data into the single tensor
    full_tensor=torch.tensor(all_values,dtype=torch.float32)
    
    # Now To feed the data to the NN we need them in the numerical foramate so need to convert features in the raw data formate (slicing them)
    raw_distance=full_tensor[:,0]
    raw_hours=full_tensor[:,1]
    raw_weekends=full_tensor[:,2]
    raw_targets=full_tensor[:,3]
    
    # We need to call the rush hour function to engineer new fetures of the RUSH HOURS
    is_rush_hour_feature=rush_hours_features(raw_hours,raw_weekends)
    
    # Unsqueeze(1) all the features to add them a new dimention 1D -> 2D
    #.unsqueeze(1)-> means add the new dimension of the 1 
    distances_col=raw_distance.unsqueeze(1)
    hours_col=raw_hours.unsqueeze(1)
    weekends_col=raw_weekends.unsqueeze(1)
    rush_hour_col=is_rush_hour_feature.unsqueeze(1)
    
    # Normalize the all the values
     # First find the mean and std values
    dist_mean,dist_std=distances_col.mean(),distances_col.std()
    hours_mean,hours_std=hours_col.mean(),hours_col.std()
    
     # Now Find the Normalize values
    dist_norm=(distances_col-dist_mean)/dist_std
    hours_norm=(hours_col-hours_mean)/hours_std
    
    # Now Concatinate all the features into the single tensor
    prepared_features=torch.cat([
        dist_norm,
        hours_norm,
        weekends_col,
        rush_hour_col
    ],dim=1)
    
    # Dimention 1 ensure that all must concatinated into the column wise features side-by-side 
    prepared_target=raw_targets.unsqueeze(1)
    result_dict={
        'full tensor':full_tensor,
        'raw distance':raw_distance,
        'raw hours':raw_hours,
        'raw weekends':raw_weekends,
        'raw targets':raw_targets,
        'distances col':distances_col,
        'hours col':hours_col,
        'weekends col':weekends_col,
        'rush hour col':rush_hour_col
    }
    norm_stats = {
        "dist_mean": dist_mean,
        "dist_std": dist_std,
        "hours_mean": hours_mean,
        "hours_std": hours_std
    }


    return prepared_features,prepared_target,norm_stats


In [84]:
# Process the entire DataFrame to get the final feature and target tensors.
feature, target,norm_stats = prepare_data(pd_read)

**Defining the Architecture of the Model**

In [85]:
def init_model():
    # Defining the Random seed To ensure the random input
    torch.manual_seed(42)
    
    # Defining the Model
    model=nn.Sequential(
        nn.Linear(4,64),
        nn.ReLU(),
        nn.Linear(64,32),
        nn.ReLU(),
        # Output Function
        nn.Linear(32,1)
    )
    optimizer=torch.optim.SGD(model.parameters(),lr=0.01)
    loss_function=nn.MSELoss()
    
    return model,optimizer,loss_function
    
 


In [86]:
model, optimizer, loss_function = init_model()

print(f"{'='*30}\nInitialized Model Architecture\n{'='*30}\n{model}")
print(f"{'='*30}\n Optimizers Optim \n{'='*30}\n{optimizer}")
print(f"{'='*30}\n Loss Function \n{'='*30}\n {loss_function}")

Initialized Model Architecture
Sequential(
  (0): Linear(in_features=4, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=1, bias=True)
)
 Optimizers Optim 
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)
 Loss Function 
 MSELoss()


*****Now Training the Model Inside the loop with the epoches*****

In [87]:
def train_model(features,targets,epochs,verbose=True):
    # First of all we need to intialize the list to store the losses
    # Here below We are gonna calculating the losses then we gonna store them inside this one list
    losses=[]



    # Now we gonna call the funtion init_model to use its features or simply we are gonna initialized
    # the model,optimizer,loss_funtion w.r.t init_model() function
    
    model,optimizer,loss_funtion=init_model()

    for epoch in range(epochs):
        # Now call the Forward pass means just passing the features through the model

        # Here the features mens the exactly the prepared_features from the earlier means the normalized values of the hours and all
        outputs=model(features)

        # Calculating the loss from the loss function
        loss=loss_function(outputs,targets)

        # Setting the Optimizers to the zero Grad means giving it the reboot
        optimizer.zero_grad()

        # Settig up the backward Function to calculate the updated values of the weights and the biases (Strictly it does not update the values Updating takes place at the optimizers step)
        loss.backward()

        # Now the Updating the Weights and the biases in the optimizer Step
        optimizer.step()

        if (epoch+1)%5000==0:
            losses.append(loss.item())
            if verbose:
                print(f"Epoch[{epoch+1}/{epochs}],Loss:{loss.item():.4f}")
    return model,losses

        
       

    


In [96]:
test_model=train_model(feature,target,10000)


Epoch[5000/10000],Loss:5.2706
Epoch[10000/10000],Loss:1.6766


In [97]:
trained_model,losses=train_model(feature,target,30000)

Epoch[5000/30000],Loss:5.2706
Epoch[10000/30000],Loss:1.6766
Epoch[15000/30000],Loss:1.5119
Epoch[20000/30000],Loss:0.7385
Epoch[25000/30000],Loss:0.3740
Epoch[30000/30000],Loss:0.4246


**Now Then Making the predication of the values****

In [98]:
def prepare_single_input(distance, hours, is_weekend, norm_stats):
    # Compute rush hour feature
    rush_hour = rush_hours_features(
        torch.tensor([hours]),
        torch.tensor([is_weekend])
    ).item()

    # Normalize using TRAINING stats
    dist_norm = (distance - norm_stats["dist_mean"]) / norm_stats["dist_std"]
    hours_norm = (hours - norm_stats["hours_mean"]) / norm_stats["hours_std"]

    # Final 2D tensor (1 sample, 4 features)
    input_tensor = torch.tensor(
        [[dist_norm, hours_norm, is_weekend, rush_hour]],
        dtype=torch.float32
    )

    return input_tensor

In [99]:
distance_miles = 15
time_of_day_hours = 14
is_weekend = 0

input_tensor = prepare_single_input(
    distance_miles,
    time_of_day_hours,
    is_weekend,
    norm_stats
)

In [100]:
with torch.no_grad():
    predicted_time = trained_model(input_tensor).item()

In [101]:
print_prediction_result(
    distance=distance_miles,
    time_str="14:00",
    is_rush_hour=False,
    predicted_time=predicted_time
)


+------------------------------------------+-----------------------+
|                         Model Prediction                         |
+------------------------------------------+-----------------------+
| Time of the Week                         | Weekday               |
| Distance                                 | 15.0 miles          |
| Time                                     | 14:00                 |
| Is this considered a rush hour period?   | No                    |
+------------------------------------------+-----------------------+
| Estimated Delivery Time                  | 39.45 minutes    |
+------------------------------------------+-----------------------+

